In [2]:
include("StackyFan.jl")

Error: Cannot load 'Ferret' due to missing binary library
Please run './configure; make' in the 'pkg/ferret' directory
#I  Getting PackageInfo URLs...
#I  Command wget not found
#I  Retrieving PackageInfo.g from https://gap-packages.github.io/ferret/PackageInfo.g ...
#I  Command wget not found
#I  Downloading archive from URL https://github.com/gap-packages/ferret/releases/download/v1.0.5/ferret-1.0.5.tar.gz ...
#I  Command wget not found
#I  Saved archive to /var/folders/pj/dxx83vnn0fjbylvczs09t5zc0000gp/T//tmm8NqIh/ferret-1.0.5.tar.gz
#I  Extracting to /Users/somethingmeaningful/.julia/gaproot/v4.11/pkg/ferret-1.0.5 ...
#I  Checking dependencies for ferret...
#I    GAPDoc >= 1.5: true
#I  Command wget not found
#I  Running compilation script on /Users/somethingmeaningful/.julia/gaproot/v4.11/pkg/ferret-1.0.5 ...
#I  Checking dependencies for ferret...
#I    GAPDoc >= 1.5: true
#I  Command wget not found
#I  Package availability test failed
#I  (for ferret 1.0.5)
#I  Removed directory

coneRayDecomposition (generic function with 1 method)

In [3]:
function getConeIndices(coneRays, rayMatrix)
    rayDict = Dict{String, Int64}()
    for i in 1:size(rayMatrix, 1)
        E=encode(rayMatrix[i,:])
        rayDict[E] = i
    end
    cone = []
    for i in 1:size(coneRays, 1)
        E = encode(coneRays[i,:])
        push!(cone, rayDict[E])
    end
    return cone
end

"""

    BerghA(F::StackyFan,D::Array{Int64,1})

    Given a stacky fan F and a vector of booleans D representing the distinguished structure,
    returns a smooth stacky fan where the distinguished rays are independent.

    
"""

function BerghA(F::StackyFan,D::Array{Int64,1},verbose::Bool=true)
    if verbose==true
        println("==algorithm is running in verbose mode==")
        println(" ")
        println("=======")
    end
    X=deepcopy(F)
    rayMatrix=convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS)))
    coneList=getCones(X.fan)
    dim=size(rayMatrix,2)
    numRays=size(rayMatrix,1)
    stackTracker=ones(Int64,numRays)
    
    #check if the vector D has length equal to the number of rays in F
    if numRays != size(D,1)
        error("length of vector representing distinguished structure does not agree with number of rays in stacky fan.")
    end
    
    #A0: initialization
    while(true)
        @time begin
            
        rayMatrix=convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS)))
        numRays=size(rayMatrix,1)
        coneList=getCones(X.fan)
        
        coneMultiplicities=Int64[]
        for cone in coneList
            C=coneConvert(cone,rayMatrix)
            push!(coneMultiplicities,coneMultiplicity(C))
        end

        #A1: check if finished
        # Find S the set of cones that contain a distinguised ray and an interior a lattice point 
        #Note: cones in S are 1-indexed.
        S=filter(cone->distinguishedAndIntPoint(cone,rayMatrix,D),coneList)
        # If S is empty, the program terminates.
        if S==[]
            break
        end
        
        #A2 - find extremal cones
        Smax=extremalCones(S,rayMatrix,D)
        if verbose==true
            Smaxcount=size(Smax,1)
            println("Number of extremal cones: $Smaxcount")
            testCone=Smax[1]
            c1=convertToIncidence(testCone,numRays)
            nonDist=size(testCone,1)-dot(c1,D)
            mult=coneMultiplicity(coneConvert(testCone,rayMatrix))
            println("Maximal non-distinguished rays and multiplicity: $nonDist, $mult")
        end
        
        
        intList = []
        for cone in Smax
            #A2 - find interior points in Smax
            intPoints=[]
            C=coneConvert(cone,rayMatrix)
            coneIntPoints=interiorPoints(C)
            if coneIntPoints==nothing
                # does the program ever reach this line if it breaks at S empty?
                return C
            end
            for point in coneIntPoints
               push!(intPoints,point) 
            end

            coneRays = rowMinors(rayMatrix,cone)
            push!(intList, (intPoints, coneRays))
        end
        

        for (intPoints, coneRays) in intList
            rayMatrix=convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS)))
            # Get cone indices from the rays
            cone = getConeIndices(coneRays, rayMatrix)
            #A2 - find stacky points (in terms of coefficients) derived from interior points
            P=Array{Int64,1}[]
            for point in intPoints
                stackyPoint=coneRayDecomposition(cone,rayMatrix,point,stackyWeights(X))
                push!(P,stackyPoint)
            end
                
            #A2 - find smallest element of P with respect to lex ordering.
            psi=minimalByLex(P)
                
            #A3 - perform root construction
            X=rootConstructionDistinguishedIndices(X,D,psi)

            #A3 - modify psi with respect to root construction
            for i in 1:length(psi)
                if D[i]==1 && psi[i]>0
                    psi[i]=1
                end
            end

            #A5 - repeat star subdivision
            while(count(x->x>0,psi)>1)
                #A4 - perform stacky star subdivision
                # Get the indices of the non-zero coefficients in psi
                supportCone=findall(x->x!=0,psi)
                #if verbose==true
                    #sW=stackyWeights(X)
                    #println("Modified stacky weights: $sW")
                #end
                exceptional=findStackyBarycenter(supportCone,X)
                #if verbose==true
                    #println("Blowing up at $exceptional")
                #end
                    
                # Convert arrays to dictionaries as constructing fan objects through subdivision
                # may reorder rays
                code_rays = mapslices(encode, Polymake.common.primitive(X.fan.RAYS), dims=2)
                # Track the indices of distinguished rays
                D_pairs = map((x,y) -> (x,y), code_rays, D)
                D_Dict = Dict(D_pairs)
                # Track psi as a linear combination of the generators
                psiPairs = map((x,y) -> (x,y), code_rays,psi)
                psiDict = Dict(psiPairs)
                trackerPairs = map((x,y) -> (x,y), code_rays, stackTracker)
                trackerDict= Dict(trackerPairs)

                X=stackyBlowup(X,[x-1 for x in supportCone],exceptional)

                G=gcd(exceptional)
                primExcep=Polymake.common.primitive(exceptional)

                # Update the dictionaries storing fan information
                D_Dict[encode(primExcep)]=1
                psiDict[encode(primExcep)]=1
                trackerDict[encode(primExcep)]=G

                newRays=slicematrix(convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS))))
                newD=Int64[]
                newpsi=Int64[]
                newTracker=Int64[]
                for ray in newRays
                    E=encode(ray)
                    excepCode=encode(primExcep)
                    push!(newD,D_Dict[E])
                    #A4 - modify psi
                    if E==excepCode
                        push!(newpsi,1)
                    elseif psiDict[E]>1
                        push!(newpsi,psiDict[E]-1)
                    else
                        push!(newpsi,0)
                    end
                    push!(newTracker,trackerDict[E])
                end
                psi=newpsi
                D=newD
                stackTracker=newTracker
                #A4 - modify psi
            end
        end
        
        end
        println("^Mainloop time")
        if verbose==true
            println("=======")
        end
    end
    
    return X, stackTracker
end

BerghA (generic function with 2 methods)

In [6]:
X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 0; 1 5],INPUT_CONES=[[0,1]])
F=addStackStructure(X,[1,1])

StackyFan(Polymake.BigObjectAllocated(Ptr{Nothing} @0x00007fbb5b946390), Dict("1,5" => 1, "1,0" => 1))

In [7]:
@time C=BerghA(F,[0,1])

==algorithm is running in verbose mode==
 
Number of extremal cones: 1
Maximal non-distinguished rays and multiplicity: 1, 5
  0.180216 seconds (10.80 k allocations: 220.698 KiB)
^Mainloop time
Number of extremal cones: 1
Maximal non-distinguished rays and multiplicity: 1, 4
  0.599082 seconds (63.29 k allocations: 878.511 KiB)
^Mainloop time
Number of extremal cones: 1
Maximal non-distinguished rays and multiplicity: 1, 2
  1.967811 seconds (296.60 k allocations: 3.521 MiB)
^Mainloop time
Number of extremal cones: 4
Maximal non-distinguished rays and multiplicity: 0, 10
  2.283467 seconds (286.48 k allocations: 4.376 MiB)
^Mainloop time
Number of extremal cones: 7
Maximal non-distinguished rays and multiplicity: 0, 5
  4.420708 seconds (557.60 k allocations: 8.086 MiB)
^Mainloop time
Number of extremal cones: 6
Maximal non-distinguished rays and multiplicity: 0, 4
  4.269912 seconds (626.46 k allocations: 8.613 MiB)
^Mainloop time
Number of extremal cones: 3
Maximal non-distinguished 

(StackyFan(Polymake.BigObjectAllocated(Ptr{Nothing} @0x00007fbb4dcb8270), Dict("18,19" => 40, "11,13" => 20, "9,10" => 20, "4,11" => 40, "22,35" => 1080, "7,8" => 10, "14,15" => 8, "3,10" => 10, "7,10" => 10, "3,4" => 10…)), [5, 1, 5, 4, 1, 20, 1, 2, 20, 1  …  20, 40, 40, 20, 40, 60, 540, 180, 540, 1080])